<a href="https://colab.research.google.com/github/mazen200/Sentiment_Analysis/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing libraries**

In [1]:
!pip install vaderSentiment
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report,f1_score,recall_score,precision_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import pickle
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier

# **Load the dataset**

In [3]:
data = pd.read_csv("/content/sentimentdataset.csv")

In [4]:
data.head()

,ID,Text,Sentiment (Label),Timestamp,User,Source,Topic,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,Enjoying a beautiful day at the park! ...,Positive,1/15/2023 12:30,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,1,Traffic was terrible this morning. ...,Negative,1/15/2023 8:45,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,2,Just finished an amazing workout! 💪 ...,Positive,1/15/2023 15:45,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,3,Excited about the upcoming weekend getaway! ...,Positive,1/15/2023 18:20,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,4,Trying out a new recipe for dinner tonight. ...,Neutral,1/15/2023 19:55,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19


# **Data Preprocessing**

In [5]:
from sklearn.preprocessing import LabelEncoder #encoding
def Feature_Encoder(X, cols):
    label_encoders = {}
    for c in cols:
        lbl = LabelEncoder()
        lbl.fit(X[c])
        X[c] = lbl.transform(X[c])
        label_encoders[c] = lbl
    return X, label_encoders

In [6]:
#one hot encoding
def Onehot_Encoder(X, cols): # -1 , 0 , 1
    onehot_encoders = {}

    # Encoding features
    for c in cols:
        # One-hot encoding
        onehot = OneHotEncoder(sparse=False, categories='auto')
        encoded = onehot.fit_transform(X[c].values.reshape(-1, 1))
        X_encoded = pd.DataFrame(encoded, columns=[c+'_'+str(i) for i in range(encoded.shape[1])])

        # Drop original column before concatenating
        X.drop(c, axis=1, inplace=True)

        # Concatenate one-hot encoded columns
        X = pd.concat([X, X_encoded], axis=1)

        onehot_encoders[c] = onehot

    return X, onehot_encoders

In [7]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
stop_words = set(stopwords.words('english'))  # Set of English stopwords
translator = str.maketrans('', '', string.punctuation)  # Translator to remove punctuation
data['Text'] = data['Text'].apply(lambda x: ' '.join([word.translate(translator) for word in x.lower().split() if word not in stop_words]))  # Remove stopwords and punctuation, and convert to lowercase

In [9]:
data.drop(columns=['User','Timestamp','Source','Retweets',	'Likes','ID',	'Country','Year','Month',	'Day','Hour'], inplace=True)

In [10]:
data.head(20)

,Text,Sentiment (Label),Topic
0,enjoying beautiful day park,Positive,#Nature #Park
1,traffic terrible morning,Negative,#Traffic #Morning
2,finished amazing workout 💪,Positive,#Fitness #Workout
3,excited upcoming weekend getaway,Positive,#Travel #Adventure
4,trying new recipe dinner tonight,Neutral,#Cooking #Food
5,feeling grateful little things life,Positive,#Gratitude #PositiveVibes
6,rainy days call cozy blankets hot cocoa,Positive,#RainyDays #Cozy
7,new movie release mustwatch,Positive,#MovieNight #MustWatch
8,political discussions heating timeline,Negative,#Politics #Debate
9,missing summer vibes beach days,Neutral,#Summer #BeachDays


In [11]:
import re

def remove_emojis(text):
    # Define a regular expression pattern to match emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Remove emojis from the text using the pattern
    return emoji_pattern.sub(r'', text)

In [12]:
data['Text'] = data['Text'].apply(remove_emojis)

In [13]:
data.head(10)

,Text,Sentiment (Label),Topic
0,enjoying beautiful day park,Positive,#Nature #Park
1,traffic terrible morning,Negative,#Traffic #Morning
2,finished amazing workout,Positive,#Fitness #Workout
3,excited upcoming weekend getaway,Positive,#Travel #Adventure
4,trying new recipe dinner tonight,Neutral,#Cooking #Food
5,feeling grateful little things life,Positive,#Gratitude #PositiveVibes
6,rainy days call cozy blankets hot cocoa,Positive,#RainyDays #Cozy
7,new movie release mustwatch,Positive,#MovieNight #MustWatch
8,political discussions heating timeline,Negative,#Politics #Debate
9,missing summer vibes beach days,Neutral,#Summer #BeachDays


In [14]:
#splitting topics
data['Topic'] = data['Topic'].str.replace('#', '')
data['Topic'] = data['Topic'].str.strip()
data[['Topic1', 'Topic2']] = data['Topic'].str.split(' ', expand=True)
data.drop(columns=['Topic'],inplace=True)

In [15]:
data.head(10)

,Text,Sentiment (Label),Topic1,Topic2
0,enjoying beautiful day park,Positive,Nature,Park
1,traffic terrible morning,Negative,Traffic,Morning
2,finished amazing workout,Positive,Fitness,Workout
3,excited upcoming weekend getaway,Positive,Travel,Adventure
4,trying new recipe dinner tonight,Neutral,Cooking,Food
5,feeling grateful little things life,Positive,Gratitude,PositiveVibes
6,rainy days call cozy blankets hot cocoa,Positive,RainyDays,Cozy
7,new movie release mustwatch,Positive,MovieNight,MustWatch
8,political discussions heating timeline,Negative,Politics,Debate
9,missing summer vibes beach days,Neutral,Summer,BeachDays


In [17]:
# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

In [18]:
# Define a function to classify text sentiment using VADER
def classify_sentiment(label):
    sentiment_score = analyzer.polarity_scores(label)
    if sentiment_score['compound'] >= 0.05:
        return 'positive'
    elif sentiment_score['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'


In [19]:
# Apply sentiment analysis to the Label column
data['Sentiment (Label)'] = data['Sentiment (Label)'].apply(classify_sentiment)

In [20]:
data.head(1000)

,Text,Sentiment (Label),Topic1,Topic2
0,enjoying beautiful day park,positive,Nature,Park
1,traffic terrible morning,negative,Traffic,Morning
2,finished amazing workout,positive,Fitness,Workout
3,excited upcoming weekend getaway,positive,Travel,Adventure
4,trying new recipe dinner tonight,neutral,Cooking,Food
...,...,...,...,...
727,collaborating science project received recogni...,positive,ScienceFairWinner,HighSchoolScience
728,attending surprise birthday party organized fr...,positive,SurpriseCelebration,HighSchoolFriendship
729,successfully fundraising school charity initia...,positive,CommunityGiving,HighSchoolPhilanthropy
730,participating multicultural festival celebrati...,positive,CulturalCelebration,HighSchoolUnity


# **Feature Engineering**

In [21]:
def REPLACE(label):
    if label == 'positive':
        return 1
    elif label == 'negative':
        return -1
    else:
        return 0

In [23]:
data['Sentiment (Label)'] = data['Sentiment (Label)'].apply(REPLACE)

In [24]:
data.head(10)

,Text,Sentiment (Label),Topic1,Topic2
0,enjoying beautiful day park,1,Nature,Park
1,traffic terrible morning,-1,Traffic,Morning
2,finished amazing workout,1,Fitness,Workout
3,excited upcoming weekend getaway,1,Travel,Adventure
4,trying new recipe dinner tonight,0,Cooking,Food
5,feeling grateful little things life,1,Gratitude,PositiveVibes
6,rainy days call cozy blankets hot cocoa,1,RainyDays,Cozy
7,new movie release mustwatch,1,MovieNight,MustWatch
8,political discussions heating timeline,-1,Politics,Debate
9,missing summer vibes beach days,0,Summer,BeachDays


In [32]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(data['Text'])  # Transform text data into numerical features
y = data['Sentiment (Label)']  # Target labels

# **Splitting data into training and testing sets**

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#**Model Selection and Training**


In [38]:
svm_model = SVC(kernel='linear')  # Using Support Vector Machine (SVM)
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

# **Model Evaluation**


In [39]:
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test,y_pred,average ='weighted')
recall = recall_score(y_test, y_pred,average ='weighted')
f1sc = f1_score(y_test, y_pred,average ='weighted')

train_acc = accuracy_score(y_train,svm_model.predict(X_train))
print("Training Accuracy:", train_acc)
# Cross-validation
cv_acc = cross_val_score(svm_model, X_train, y_train, cv=5)
print("Cross-validation Accuracies:", cv_acc)

print("test Accuracy:", accuracy)
print("precision : " , precision)
print("recall : ", recall)
print("f1 score : ",f1_score )
#print("Classification Report:")
#print(classification_report(y_test, y_pred))
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)
input_text = ["mohamed drink coffee"]

# Preprocess the input text
preprocessed_text = [preprocess_text(text) for text in input_text]

# Convert preprocessed text into TF-IDF vectors
input_text_tfidf = tfidf_vectorizer.transform(preprocessed_text)

# Predict sentiment label
prediction = svm_model.predict(input_text_tfidf)

print("Predicted Sentiment Label:", prediction)
#decoded_prediction = label_encoders["Sentiment (Label)"].inverse_transform(prediction)
#print(decoded_prediction[0])

Training Accuracy: 0.9811965811965812
Cross-validation Accuracies: [0.69230769 0.76068376 0.74358974 0.73504274 0.73504274]
test Accuracy: 0.7210884353741497
precision :  0.7947333772218564
recall :  0.7210884353741497
f1 score :  <function f1_score at 0x7db506b8a440>
Predicted Sentiment Label: [1]


In [ ]:
#pickle.dump(svm_model, open('trained_model.sav', 'wb'))
#pickle.dump(tfidf_vectorizer, open('vectorized_model.sav', 'wb'))
#pickle.dump(label_encoders, open('label_encoder.sav', 'wb'))